In [16]:
import pandas as pd
import numpy as np

n_bins = 4
eps = 1e-6
label_col = "survival_months"

TARGET = "ucec"
TARGET_FILE = f"tcga_{TARGET}_all.csv"

df = pd.read_csv(TARGET_FILE)
df.case_id = df.case_id.map(lambda x: x[:12])
df.slide_id = df.slide_id.map(lambda x: x.split(".")[0] + "." + x.split(".")[2])

patients_df = df.drop_duplicates(['case_id']).copy()
uncensored_df = patients_df[patients_df['censorship'] < 1]

disc_labels, q_bins = pd.qcut(uncensored_df[label_col], q=n_bins, retbins=True, labels=False)
q_bins[-1] = df[label_col].max() + eps
q_bins[0] = df[label_col].min() - eps

disc_labels, q_bins = pd.cut(patients_df[label_col], bins=q_bins, retbins=True, labels=False, right=False, include_lowest=True)

patients_df.loc[:, "label"] = disc_labels.values.astype(int)
patients_df.to_csv(TARGET_FILE, index=False)